In [1]:
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import pandas as pd

In [2]:
df = pd.read_csv(r"C:\Users\johna\anaconda3\envs\twitter-analytics-env\twitter_issues_dashboard\twitter_issues_dashboard\data\01_raw\tweets_details2023-03-15_20-43-36.csv")

In [16]:
def clean_text(text):
    # convert input to str
    text = str(text)
    
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    
    # Remove special characters
    text = re.sub(r'[^\w\s]', '', text)
    
    # Remove ASCII characters
    text = re.sub(r'[^\x00-\x7f]', '', text)
    
    # Remove @ symbols and # symbols
    text = re.sub(r'[@#]\w+', '', text)
    
    # Remove variations of "royal mail"
    text = re.sub(r'royal\s*mail', '', text, flags=re.IGNORECASE)
    
    # Tokenize the text
    tokens = word_tokenize(text)
    
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token.lower() not in stop_words]
    
    # Lemmatize the tokens
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    # Join the cleaned tokens back into a string
    cleaned_text = ' '.join(tokens)
    
    return cleaned_text


# Apply the clean_text function to the 'text' column in your DataFrame
df['cleaned_text'] = df['text'].apply(clean_text)


In [13]:
df.head(5)

tweet_id                   creation_date  \
0  7.485231e+17  Thu Jun 30 14:26:32 +0000 2016   
1  7.484731e+17  Thu Jun 30 11:07:57 +0000 2016   
2  7.485466e+17  Thu Jun 30 15:59:48 +0000 2016   
3  7.485004e+17  Thu Jun 30 12:56:24 +0000 2016   
4  7.485809e+17  Thu Jun 30 18:16:08 +0000 2016   

                                                text  \
0  #royalmail offers Compulsory new “Chargeable” ...   
1  Great #friends realize your #love of all thing...   
2  My #dilevery of #easyskinz with #royalmail #ip...   
3  #royalmail ridiculous rule not being able to s...   
4  📮 #RoyalMail MarketReach Report Dispels #Direc...   

                                           media_url video_url language  \
0                                                NaN       NaN       en   
1                                                NaN       NaN       en   
2                                                NaN       NaN       en   
3                                                NaN       NaN       en   
4  ['https://pbs.twimg.com/media/CmL0Z4qWIAAJDDe....       NaN       en   

   favorite_count  retweet_count  reply_count  quote_count  ...  \
0             0.0            0.0          0.0          0.0  ...   
1             1.0            0.0          0.0          0.0  ...   
2             1.0            0.0          0.0          0.0  ...   
3             0.0            0.0          0.0          0.0  ...   
4             1.0            0.0          0.0          0.0  ...   

                             user.profile_banner_url  \
0  https://pbs.twimg.com/profile_banners/29701699...   
1  https://pbs.twimg.com/profile_banners/14168888...   
2  https://pbs.twimg.com/profile_banners/20078001...   
3  https://pbs.twimg.com/profile_banners/24632482...   
4  https://pbs.twimg.com/profile_banners/30078697...   

                                    user.description  \
0  Selro provides all the tools to start and grow...   
1                                                NaN   
2                                instagram: thimothy   
3  Consumer rights activist— holding companies to...   
4  #UK based #printing company.         https://t...   

                         user.external_url  user.number_of_tweets  user.bot  \
0                     http://www.selro.com                 1036.0     False   
1  http://our-labour-of-love.blogspot.com/                 1973.0     False   
2                                      NaN                 1117.0     False   
3                                      NaN                16688.0     False   
4                 http://www.printmr.co.uk                 9769.0     False   

   user.timestamp user.has_nft_avatar user detail  \
0    1.239193e+09               False  NaN    NaN   
1    1.368150e+09               False  NaN    NaN   
2    1.286693e+09               False  NaN    NaN   
3    1.398434e+09               False  NaN    NaN   
4    1.305716e+09               False  NaN    NaN   

                                        cleaned_text  
0  offer Compulsory new Chargeable IntegratorCont...  
1  Great friend realize love thing written Love g...  
2  dilevery easyskinz iphone6 mehappy blackmamba ...  
3  ridiculous rule able send perfume etc post eve...  
4  MarketReach Report Dispels DirectMail Engageme...  

[5 rows x 40 columns]

In [24]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# Create a CountVectorizer object to convert the cleaned text into a bag-of-words representation
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['cleaned_text'])

# Train the LDA model
lda_model = LatentDirichletAllocation(n_components=200, random_state=42)
lda_model.fit(X)

# Get the feature names from the vectorizer
feature_names = vectorizer.get_feature_names_out()

topic_keywords = []

# Print the top 10 words for each topic
for topic_idx, topic in enumerate(lda_model.components_):
    print("Topic %d:" % (topic_idx))
    top_features = [feature_names[i] for i in topic.argsort()[:-11:-1]]
    print(", ".join(top_features))
    top_words = [feature_names[i] for i in topic.argsort()[:-11:-1]]
    topic_keywords.append(', '.join(top_words))

# add topic keywords to dataframe
df['topic_keywords'] = [topic_keywords[i] for i in lda_model.transform(vectorizer.transform(df['cleaned_text'])).argmax(axis=1)]



Topic 0:
postcard, snailmail, postcrossing, post, pillarbox, mail, ii, england, postbox, via
Topic 1:
woman, visit, low, premium, amp, share, rmgl, livelihood, shipment, basic
Topic 2:
eu, law, charged, refund, breaking, vat, record, service, serious, seller
Topic 3:
buy, usps, sell, stock, rmg, share, ftse, trading, investing, amp
Topic 4:
safe, place, secure, post, charles, keep, see, delivery, uk, everyone
Topic 5:
edinburgh, force, self, daily, parcel, postoffice, post, employed, onto, tweet
Topic 6:
detail, account, explain, asking, help, caught, please, parcel, genuine, ebay
Topic 7:
trip, wondering, bus, post, parcel, today, round, mail, im, mile
Topic 8:
state, post, scams, private, money, yep, wondered, finished, year, paid
Topic 9:
scam, parcelforce, delivery, email, phishing, hermesparcels, customer, fake, warning, urgent
Topic 10:
water, energy, amp, barcoded, labour, nhs, rail, privatisation, nationalise, uklabour
Topic 11:
covid19, coronavirus, staff, crisis, test, uk, nh

In [23]:
df

tweet_id                   creation_date  \
0      7.485231e+17  Thu Jun 30 14:26:32 +0000 2016   
1      7.484731e+17  Thu Jun 30 11:07:57 +0000 2016   
2      7.485466e+17  Thu Jun 30 15:59:48 +0000 2016   
3      7.485004e+17  Thu Jun 30 12:56:24 +0000 2016   
4      7.485809e+17  Thu Jun 30 18:16:08 +0000 2016   
...             ...                             ...   
31541  1.635522e+18  Tue Mar 14 06:02:30 +0000 2023   
31542  1.635658e+18  Tue Mar 14 15:02:41 +0000 2023   
31543  1.635549e+18  Tue Mar 14 07:51:07 +0000 2023   
31544  1.635681e+18  Tue Mar 14 16:36:46 +0000 2023   
31545  1.635506e+18  Tue Mar 14 04:58:28 +0000 2023   

                                                    text  \
0      #royalmail offers Compulsory new “Chargeable” ...   
1      Great #friends realize your #love of all thing...   
2      My #dilevery of #easyskinz with #royalmail #ip...   
3      #royalmail ridiculous rule not being able to s...   
4      📮 #RoyalMail MarketReach Report Dispels #Direc...   
...                                                  ...   
31541  Good morning! We’ll be open from 7am for all o...   
31542  Royaume-Uni : Royal Mail Émet Pour La Première...   
31543  Yes, also think trying to find dentist accepti...   
31544  #Postiepics #postie #postman #royalmail #myroy...   
31545  The UK's #RoyalMail has unveiled the first set...   

                                               media_url video_url language  \
0                                                    NaN       NaN       en   
1                                                    NaN       NaN       en   
2                                                    NaN       NaN       en   
3                                                    NaN       NaN       en   
4      ['https://pbs.twimg.com/media/CmL0Z4qWIAAJDDe....       NaN       en   
...                                                  ...       ...      ...   
31541  ['https://pbs.twimg.com/media/FrKKYKHX0AA16KB....       NaN       en   
31542                                                NaN       NaN       fr   
31543                                                NaN       NaN       en   
31544  ['https://pbs.twimg.com/media/FrMbjLHWYAAPNSP....       NaN      qme   
31545                                                NaN       NaN       en   

       favorite_count  retweet_count  reply_count  quote_count  ...  \
0                 0.0            0.0          0.0          0.0  ...   
1                 1.0            0.0          0.0          0.0  ...   
2                 1.0            0.0          0.0          0.0  ...   
3                 0.0            0.0          0.0          0.0  ...   
4                 1.0            0.0          0.0          0.0  ...   
...               ...            ...          ...          ...  ...   
31541             9.0            3.0          1.0          0.0  ...   
31542             0.0            0.0          0.0          0.0  ...   
31543             0.0            0.0          0.0          0.0  ...   
31544             0.0            0.0          0.0          0.0  ...   
31545             0.0            0.0          0.0          0.0  ...   

                                        user.description  \
0      Selro provides all the tools to start and grow...   
1                                                    NaN   
2                                    instagram: thimothy   
3      Consumer rights activist— holding companies to...   
4      #UK based #printing company.         https://t...   
...                                                  ...   
31541  Convenience Store & Post Office Selling 🗞, 🍞,🍏...   
31542  Le Matinal est la principale plate-forme média...   
31543  MrStevePassmoor01/08/09 MumOf4&GrandmaTo1 From...   
31544            Pretty good. Not bad. I can't complain.   
31545  Your daily supplement of news, facts, analysis...   

                             user.external_url  user.number_of_tweets  \
0                         http://